In [ ]:
from datasets import load_dataset
dataset = load_dataset("keremberke/chest-xray-classification",'full')
dataset.save_to_disk("./chest_xray_clasnsificatio")

In [ ]:
from datasets import DatasetDict, concatenate_datasets

dataset = DatasetDict.load_from_disk("./chest_xray_clasnsificatio")

def select_samples(dataset, label_value, num_samples):
    filtered = dataset.filter(lambda example: example['labels'] == label_value)
    return filtered.select(range(min(num_samples, len(filtered))))

def create_balanced_subset(dataset, num_samples=100):
    balanced_dataset = {}
    for split in dataset.keys():
        label_0_samples = select_samples(dataset[split], label_value=0, num_samples=num_samples)
        label_1_samples = select_samples(dataset[split], label_value=1, num_samples=num_samples)
        balanced_dataset[split] = concatenate_datasets([label_0_samples, label_1_samples])
    return DatasetDict(balanced_dataset)

balanced_dataset = create_balanced_subset(dataset, num_samples=100)

print(balanced_dataset)


Filter:   0%|          | 0/1165 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1165 [00:00<?, ? examples/s]

Filter:   0%|          | 0/582 [00:00<?, ? examples/s]

Filter:   0%|          | 0/582 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 200
    })
    test: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 200
    })
})


In [ ]:
import torch
import torch.nn as nn
import accelerate
from torch.utils.data import DataLoader
from torchvision.transforms import (
    Compose, Resize, ToTensor, Normalize
)
from transformers import ViTModel,ViTConfig, ViTForImageClassification, Trainer, TrainingArguments
from datasets import DatasetDict
dataset=balanced_dataset
image_size = 224
transforms = Compose([
    Resize((image_size, image_size)),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def preprocess_data(example):
    example['pixel_values'] = transforms(example['image'])
    return example

dataset = dataset.map(preprocess_data, batched=False)

dataset = dataset.remove_columns(["image_file_path", "image"])

print(dataset)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 200
    })
    validation: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 200
    })
    test: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 200
    })
})


In [ ]:
import torch
import torch.nn as nn
from transformers import ViTConfig, ViTForImageClassification
from transformers import Trainer, TrainingArguments
from datasets import DatasetDict


class ConvStem(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=7, stride=2, padding=3, output_size=None):
        super(ConvStem, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.output_size = output_size 

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.pool(x)
        if self.output_size:
            x = nn.functional.interpolate(x, size=self.output_size, mode='bilinear', align_corners=False)
        return x


class ViTWithConvStem(nn.Module):
    def __init__(self, config, conv_out_channels=64):
        super().__init__()
        patch_size = config.patch_size
        image_size = config.image_size
        aligned_size = (image_size // patch_size) * patch_size

        self.conv_stem = ConvStem(
            in_channels=config.num_channels,
            out_channels=conv_out_channels,
            kernel_size=7,
            stride=2,
            padding=3,
            output_size=(aligned_size, aligned_size), 
        )
        self.vit = ViTForImageClassification(config)

    def forward(self, pixel_values, labels=None):
        x = self.conv_stem(pixel_values)

        batch_size, channels, height, width = x.shape
        patch_size = self.vit.config.patch_size

        x = x.unfold(2, patch_size, patch_size).unfold(3, patch_size, patch_size)
        x = x.permute(0, 2, 3, 1, 4, 5).contiguous()
        x = x.view(batch_size, -1, patch_size * patch_size * channels)

        outputs = self.vit(pixel_values=pixel_values, labels=labels)
        return outputs


config = ViTConfig(
    image_size=224,
    num_channels=3,
    patch_size=16,
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    intermediate_size=3072,
    num_labels=2
)


model = ViTWithConvStem(config)

training_args = TrainingArguments(
    output_dir="./vit_model_1201",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)

from evaluate import load
metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions.numpy(), references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=None,
    compute_metrics=compute_metrics,
)

trainer.train()
results = trainer.evaluate(dataset["test"])
print("Test Results:", results)


C:\Users\19811\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\19811\AppData\Local\Temp\ipykernel_20636\2240434196.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/130 [00:00<?, ?it/s]

{'loss': 1.1706, 'grad_norm': 4.27631950378418, 'learning_rate': 4.615384615384616e-05, 'epoch': 0.77}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.7703848481178284, 'eval_accuracy': 0.5, 'eval_runtime': 28.7309, 'eval_samples_per_second': 6.961, 'eval_steps_per_second': 0.452, 'epoch': 1.0}
{'loss': 0.7787, 'grad_norm': 12.118109703063965, 'learning_rate': 4.230769230769231e-05, 'epoch': 1.54}


...

{'eval_loss': 0.6674125790596008, 'eval_accuracy': 0.78, 'eval_runtime': 29.1973, 'eval_samples_per_second': 6.85, 'eval_steps_per_second': 0.445, 'epoch': 9.0}
{'loss': 0.1964, 'grad_norm': 21.13983917236328, 'learning_rate': 3.846153846153847e-06, 'epoch': 9.23}
{'loss': 0.2211, 'grad_norm': 25.287439346313477, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.4938441514968872, 'eval_accuracy': 0.84, 'eval_runtime': 29.3959, 'eval_samples_per_second': 6.804, 'eval_steps_per_second': 0.442, 'epoch': 10.0}
{'train_runtime': 942.5098, 'train_samples_per_second': 2.122, 'train_steps_per_second': 0.138, 'train_loss': 0.5300321899927579, 'epoch': 10.0}


  0%|          | 0/13 [00:00<?, ?it/s]

Test Results: {'eval_loss': 0.716962993144989, 'eval_accuracy': 0.805, 'eval_runtime': 28.9942, 'eval_samples_per_second': 6.898, 'eval_steps_per_second': 0.448, 'epoch': 10.0}


In [8]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

training_args = TrainingArguments(
    output_dir="./vit_model_1201",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    learning_rate=4e-6,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=None,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

trainer.train()
results = trainer.evaluate(dataset["test"])
print("Test Results:", results)

C:\Users\19811\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\19811\AppData\Local\Temp\ipykernel_20636\1662128200.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/195 [00:00<?, ?it/s]

{'loss': 0.2072, 'grad_norm': 3.23065447807312, 'learning_rate': 3.7948717948717945e-06, 'epoch': 0.77}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.4808088541030884, 'eval_accuracy': 0.84, 'eval_runtime': 29.1606, 'eval_samples_per_second': 6.859, 'eval_steps_per_second': 0.446, 'epoch': 1.0}
{'loss': 0.1822, 'grad_norm': 7.942155838012695, 'learning_rate': 3.5897435897435896e-06, 'epoch': 1.54}


...

{'eval_loss': 0.5948677062988281, 'eval_accuracy': 0.855, 'eval_runtime': 28.3314, 'eval_samples_per_second': 7.059, 'eval_steps_per_second': 0.459, 'epoch': 6.0}
{'loss': 0.1151, 'grad_norm': 14.335393905639648, 'learning_rate': 2.358974358974359e-06, 'epoch': 6.15}
{'loss': 0.1367, 'grad_norm': 2.18445086479187, 'learning_rate': 2.1538461538461538e-06, 'epoch': 6.92}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.6589303016662598, 'eval_accuracy': 0.845, 'eval_runtime': 29.1401, 'eval_samples_per_second': 6.863, 'eval_steps_per_second': 0.446, 'epoch': 7.0}
{'train_runtime': 656.003, 'train_samples_per_second': 4.573, 'train_steps_per_second': 0.297, 'train_loss': 0.1633449954768786, 'epoch': 7.0}


  0%|          | 0/13 [00:00<?, ?it/s]

Test Results: {'eval_loss': 0.8433399200439453, 'eval_accuracy': 0.815, 'eval_runtime': 28.5323, 'eval_samples_per_second': 7.01, 'eval_steps_per_second': 0.456, 'epoch': 7.0}


In [ ]:
dataset = DatasetDict.load_from_disk("./chest_xray_clasnsificatio")

image_size = 224
transforms = Compose([
    Resize((image_size, image_size)),
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def preprocess_data(example):
    example['pixel_values'] = transforms(example['image'])
    return example

dataset = dataset.map(preprocess_data, batched=False)

dataset = dataset.remove_columns(["image_file_path", "image"])

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 4077
    })
    validation: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 1165
    })
    test: Dataset({
        features: ['labels', 'pixel_values'],
        num_rows: 582
    })
})


In [10]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=None,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

trainer.train()
results = trainer.evaluate(dataset["test"])
print("Test Results:", results)



C:\Users\19811\AppData\Local\Temp\ipykernel_20636\4158758695.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/3825 [00:00<?, ?it/s]

{'loss': 0.5214, 'grad_norm': 10.320158004760742, 'learning_rate': 3.989542483660131e-06, 'epoch': 0.04}
{'loss': 0.457, 'grad_norm': 13.489970207214355, 'learning_rate': 3.979084967320261e-06, 'epoch': 0.08}
{'loss': 0.3261, 'grad_norm': 5.10440731048584, 'learning_rate': 3.968627450980392e-06, 'epoch': 0.12}
{'loss': 0.2952, 'grad_norm': 4.566559791564941, 'learning_rate': 3.958169934640523e-06, 'epoch': 0.16}
{'loss': 0.4452, 'grad_norm': 8.832679748535156, 'learning_rate': 3.947712418300654e-06, 'epoch': 0.2}
{'loss': 0.2808, 'grad_norm': 14.268045425415039, 'learning_rate': 3.937254901960784e-06, 'epoch': 0.24}
{'loss': 0.3413, 'grad_norm': 3.437281608581543, 'learning_rate': 3.9267973856209145e-06, 'epoch': 0.27}
{'loss': 0.3436, 'grad_norm': 3.749412775039673, 'learning_rate': 3.916339869281045e-06, 'epoch': 0.31}
{'loss': 0.3351, 'grad_norm': 4.614947319030762, 'learning_rate': 3.905882352941176e-06, 'epoch': 0.35}
{'loss': 0.3635, 'grad_norm': 6.122328758239746, 'learning_rate

  0%|          | 0/73 [00:00<?, ?it/s]

{'eval_loss': 0.2900601327419281, 'eval_accuracy': 0.8755364806866953, 'eval_runtime': 166.9877, 'eval_samples_per_second': 6.977, 'eval_steps_per_second': 0.437, 'epoch': 1.0}
{'loss': 0.2952, 'grad_norm': 10.296650886535645, 'learning_rate': 3.7281045751633987e-06, 'epoch': 1.02}
{'loss': 0.2965, 'grad_norm': 2.2865076065063477, 'learning_rate': 3.7176470588235295e-06, 'epoch': 1.06}
{'loss': 0.4124, 'grad_norm': 24.955238342285156, 'learning_rate': 3.7071895424836603e-06, 'epoch': 1.1}
{'loss': 0.2849, 'grad_norm': 6.959079265594482, 'learning_rate': 3.696732026143791e-06, 'epoch': 1.14}
{'loss': 0.2876, 'grad_norm': 11.02238655090332, 'learning_rate': 3.686274509803921e-06, 'epoch': 1.18}
{'loss': 0.3225, 'grad_norm': 10.804498672485352, 'learning_rate': 3.675816993464052e-06, 'epoch': 1.22}
{'loss': 0.3173, 'grad_norm': 13.52967643737793, 'learning_rate': 3.6653594771241826e-06, 'epoch': 1.25}
{'loss': 0.2775, 'grad_norm': 16.68783187866211, 'learning_rate': 3.6549019607843134e-06

...

{'eval_loss': 0.23749525845050812, 'eval_accuracy': 0.9356223175965666, 'eval_runtime': 181.017, 'eval_samples_per_second': 6.436, 'eval_steps_per_second': 0.403, 'epoch': 14.0}
{'loss': 0.1755, 'grad_norm': 13.70283317565918, 'learning_rate': 2.5620915032679736e-07, 'epoch': 14.04}
{'loss': 0.1464, 'grad_norm': 47.662940979003906, 'learning_rate': 2.457516339869281e-07, 'epoch': 14.08}
{'loss': 0.0998, 'grad_norm': 3.9633285999298096, 'learning_rate': 2.352941176470588e-07, 'epoch': 14.12}
{'loss': 0.1944, 'grad_norm': 41.443214416503906, 'learning_rate': 2.2483660130718955e-07, 'epoch': 14.16}
{'loss': 0.1429, 'grad_norm': 85.75756072998047, 'learning_rate': 2.1437908496732023e-07, 'epoch': 14.2}
{'loss': 0.159, 'grad_norm': 3.4543466567993164, 'learning_rate': 2.0392156862745097e-07, 'epoch': 14.24}
{'loss': 0.2513, 'grad_norm': 4.192806243896484, 'learning_rate': 1.934640522875817e-07, 'epoch': 14.27}
{'loss': 0.1592, 'grad_norm': 101.54142761230469, 'learning_rate': 1.830065359477

  0%|          | 0/73 [00:00<?, ?it/s]

{'eval_loss': 0.2497621476650238, 'eval_accuracy': 0.9339055793991416, 'eval_runtime': 169.7735, 'eval_samples_per_second': 6.862, 'eval_steps_per_second': 0.43, 'epoch': 15.0}
{'train_runtime': 22380.228, 'train_samples_per_second': 2.733, 'train_steps_per_second': 0.171, 'train_loss': 0.20743973994371936, 'epoch': 15.0}


  0%|          | 0/37 [00:00<?, ?it/s]

Test Results: {'eval_loss': 0.20613574981689453, 'eval_accuracy': 0.936426116838488, 'eval_runtime': 86.1279, 'eval_samples_per_second': 6.757, 'eval_steps_per_second': 0.43, 'epoch': 15.0}


In [11]:
torch.save(model.state_dict(), 'model_parameters_1201.pth')

In [12]:
training_args = TrainingArguments(
    output_dir="./vit_model_1201",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    learning_rate=5e-9,
    weight_decay=0.01,
    save_total_limit=2,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=None,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping],
)

trainer.train()
results = trainer.evaluate(dataset["test"])
print("Test Results:", results)

C:\Users\19811\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\19811\AppData\Local\Temp\ipykernel_20636\2839116516.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/5100 [00:00<?, ?it/s]

{'loss': 0.0962, 'grad_norm': 0.6237013339996338, 'learning_rate': 4.990196078431373e-09, 'epoch': 0.04}
{'loss': 0.0526, 'grad_norm': 1.2592545747756958, 'learning_rate': 4.980392156862745e-09, 'epoch': 0.08}
{'loss': 0.1246, 'grad_norm': 12.886646270751953, 'learning_rate': 4.970588235294118e-09, 'epoch': 0.12}
{'loss': 0.1764, 'grad_norm': 1.2968604564666748, 'learning_rate': 4.96078431372549e-09, 'epoch': 0.16}
{'loss': 0.1424, 'grad_norm': 20.541128158569336, 'learning_rate': 4.950980392156863e-09, 'epoch': 0.2}
{'loss': 0.0992, 'grad_norm': 22.400251388549805, 'learning_rate': 4.941176470588235e-09, 'epoch': 0.24}
{'loss': 0.1788, 'grad_norm': 6.979392051696777, 'learning_rate': 4.931372549019608e-09, 'epoch': 0.27}
{'loss': 0.1596, 'grad_norm': 13.072405815124512, 'learning_rate': 4.92156862745098e-09, 'epoch': 0.31}
{'loss': 0.0989, 'grad_norm': 3.2910754680633545, 'learning_rate': 4.9117647058823525e-09, 'epoch': 0.35}
{'loss': 0.1727, 'grad_norm': 70.84709930419922, 'learning

  0%|          | 0/73 [00:00<?, ?it/s]

{'eval_loss': 0.2209848314523697, 'eval_accuracy': 0.9381974248927039, 'eval_runtime': 174.8566, 'eval_samples_per_second': 6.663, 'eval_steps_per_second': 0.417, 'epoch': 1.0}
{'loss': 0.159, 'grad_norm': 50.63440704345703, 'learning_rate': 4.745098039215686e-09, 'epoch': 1.02}
{'loss': 0.0831, 'grad_norm': 2.2748358249664307, 'learning_rate': 4.735294117647059e-09, 'epoch': 1.06}
{'loss': 0.1749, 'grad_norm': 82.53803253173828, 'learning_rate': 4.725490196078431e-09, 'epoch': 1.1}
{'loss': 0.1076, 'grad_norm': 23.359214782714844, 'learning_rate': 4.715686274509804e-09, 'epoch': 1.14}
{'loss': 0.1487, 'grad_norm': 4.227215766906738, 'learning_rate': 4.705882352941176e-09, 'epoch': 1.18}
{'loss': 0.1612, 'grad_norm': 41.28520202636719, 'learning_rate': 4.696078431372549e-09, 'epoch': 1.22}
{'loss': 0.1064, 'grad_norm': 28.58900260925293, 'learning_rate': 4.686274509803922e-09, 'epoch': 1.25}
{'loss': 0.1697, 'grad_norm': 4.768855571746826, 'learning_rate': 4.676470588235294e-09, 'epoch

...

{'eval_loss': 0.2239675670862198, 'eval_accuracy': 0.9356223175965666, 'eval_runtime': 172.7223, 'eval_samples_per_second': 6.745, 'eval_steps_per_second': 0.423, 'epoch': 3.0}
{'loss': 0.0867, 'grad_norm': 22.529979705810547, 'learning_rate': 4.2450980392156865e-09, 'epoch': 3.02}
{'loss': 0.1713, 'grad_norm': 11.340468406677246, 'learning_rate': 4.235294117647059e-09, 'epoch': 3.06}
{'loss': 0.0787, 'grad_norm': 15.864761352539062, 'learning_rate': 4.2254901960784315e-09, 'epoch': 3.1}
{'loss': 0.1989, 'grad_norm': 47.87004470825195, 'learning_rate': 4.215686274509804e-09, 'epoch': 3.14}
{'loss': 0.1534, 'grad_norm': 58.8902473449707, 'learning_rate': 4.205882352941177e-09, 'epoch': 3.18}
{'loss': 0.1157, 'grad_norm': 30.273406982421875, 'learning_rate': 4.1960784313725496e-09, 'epoch': 3.22}
{'loss': 0.1348, 'grad_norm': 23.036636352539062, 'learning_rate': 4.186274509803922e-09, 'epoch': 3.25}
{'loss': 0.1351, 'grad_norm': 0.27133795619010925, 'learning_rate': 4.176470588235295e-09

  0%|          | 0/73 [00:00<?, ?it/s]

{'eval_loss': 0.2247777283191681, 'eval_accuracy': 0.9356223175965666, 'eval_runtime': 174.3097, 'eval_samples_per_second': 6.684, 'eval_steps_per_second': 0.419, 'epoch': 4.0}
{'train_runtime': 6085.9954, 'train_samples_per_second': 13.398, 'train_steps_per_second': 0.838, 'train_loss': 0.1472910403694008, 'epoch': 4.0}


  0%|          | 0/37 [00:00<?, ?it/s]

Test Results: {'eval_loss': 0.19924920797348022, 'eval_accuracy': 0.9415807560137457, 'eval_runtime': 88.2956, 'eval_samples_per_second': 6.591, 'eval_steps_per_second': 0.419, 'epoch': 4.0}


In [13]:
torch.save(model.state_dict(), 'model_parameters_1202.pth')